In [1]:
# Let's load the precompuded differential data
include("../differentials/computation/sl3_utils.jl");

  Activating project at `~/Desktop/LowCohomologySOS`
[ Info: Precompiling LowCohomologySOS [70c9f613-c550-58ca-b626-6da403f9846d]


In [ ]:
# Sanity checks for vanishing compositions of differentials

In [2]:
d₄*d₅ == reshape([zero(RG); zero(RG)],2,1)

true

In [6]:
d₃*d₄ == [zero(RG)]

true

# Computing Laplacians

In [10]:
# The stabiliser parts which we have to add to get free modules.
# Hopefully (and quite suprisingly for me) the stabilisers' elements belong to half_basis.
dim2_stab_part = reshape([one(RG)-averaged_rep(m2_stab, half_basis, RG)], 1, 1)
dim3_stab_part = [
    one(RG)-averaged_rep(m31_stab, half_basis, RG) zero(RG);
    zero(RG) one(RG)-averaged_rep(m32_stab, half_basis, RG)
]
dim4_stab_part = reshape([one(RG)-averaged_rep(m4_stab, half_basis, RG)], 1, 1)
dim5_stab_part = reshape([one(RG)-averaged_rep(m5_stab, half_basis, RG)], 1, 1);

In [17]:
# Compute the Laplacians.
# At the end, we embed the Laplacian into RG_star, the group ring
# with the same basis as RG but with twisted multiplciation, i.e.
# (1+g)(1+h)=1+g+h+g^(-1)h. This is needed to translate hermitian squares
# to the standard ones for the definition of the semi-definite optimization problem
# (solvers prefer standard squares to hermitian :).
# This has no effect on the shape of the Laplacian since we just embed it.
RG_star = LowCohomologySOS.group_ring(sl3, half_basis, star_multiplication = true)

Δ₂x = d₃*d₃'+dim2_stab_part
Δ₃x = d₃'*d₃+d₄*d₄'+dim3_stab_part
Δ₄x = reshape([d₄'*d₄],1,1)+d₅*d₅'+dim4_stab_part
Δ₅x = d₅'*d₅+dim5_stab_part

Δ₂ = LowCohomologySOS.embed.(identity, Δ₂x, Ref(RG_star))
Δ₃ = LowCohomologySOS.embed.(identity, Δ₃x, Ref(RG_star))
Δ₄ = LowCohomologySOS.embed.(identity, Δ₄x, Ref(RG_star))
Δ₅ = LowCohomologySOS.embed.(identity, Δ₅x, Ref(RG_star));

In [18]:
# Check if the Laplacians are hermitian (just to be sure we have not spolied something obvious)
@info Δ₂' == Δ₂
@info Δ₃' == Δ₃
@info Δ₄' == Δ₄
@info Δ₅' == Δ₅

[ Info: true
[ Info: true
[ Info: true
[ Info: true


In [20]:
# Load the solver ...
include("/home/mizerka/Desktop/LowCohomologySOS/scripts/optimizers.jl");

[ Info: Precompiling SCS [c946c3f1-0d1f-5ce8-9dea-7daa1f7e2d13]


In [21]:
# ... and the optimization package
using JuMP

# Finding spectral gaps

We want to find a numerical approximation: λ and Q such that Δₙ-λI ≈ 𝕩*QᵀQ𝕩,
where 𝕩 is the vector formed from half_basis as in https://arxiv.org/abs/2207.02783, Lemma 3.3.

Since H⁰ and H³ don't vanish for every unitary rep., there mustn't be spectral gaps for these Laplacians.
Since SL(3,ℤ) has property (T), we hope to find a spectral gap for Δ₄.
Finally, we want to prove H² = 0, that is find a spectral gap for Δ₃


## Δ₂

In [58]:
I = reshape([one(RG_star)],1,1)
sos_problem = LowCohomologySOS.sos_problem(Δ₂, I, 0.05) # on the cost of optimality, bound the gap frome above for a quicker solution
JuMP.set_optimizer(sos_problem, scs_opt(eps = 1e-9, max_iters = 20_000))
JuMP.optimize!(sos_problem)

------------------------------------------------------------------
	       SCS v3.2.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 50722, constraints m: 57536
cones: 	  z: primal zero / dual free vars: 6814
	  l: linear vars: 1
	  s: psd vars: 50721, ssize: 1
settings: eps_abs: 1.0e-09, eps_rel: 1.0e-09, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 20000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 148617, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 2.00e+01  1.00e+00  2.00e+01 -1.01e+01  1.00e-01  1.39e-01 
   250| 5.80e-06  2.60e-07  7.67e-05 -5.00e-02  5.73e+00  1.13e

In [59]:
# Certify the solution.
# For the certification theoretical background, see https://arxiv.org/abs/2207.02783, Proposition 3.2.
λ, Q = LowCohomologySOS.get_solution(sos_problem)
LowCohomologySOS.certify_sos_decomposition(Δ₂, I, λ, Q, half_basis)

┌ Info: l₁ norm of the error in interval arithmetic:
│   l1_norm = [2.73146e-07, 2.73149e-07]
└   radius(l1_norm) = 6.624725319600648e-13


(false, [-2.7339e-07, -2.73387e-07])

## Δ₃

It looks that we are unlucky - for the chosen support (i.e. half_basis defined by differentials), there is no spectral gap: λ = 0. Maybe the second cohomology is not vanishing for every unitary rep.?

In [56]:
I = [
    one(RG_star) zero(RG_star);
    zero(RG_star) one(RG_star)
]
sos_problem = LowCohomologySOS.sos_problem(Δ₃, I, 0.05)
JuMP.set_optimizer(sos_problem, scs_opt(eps = 1e-9, max_iters = 20_000))
JuMP.optimize!(sos_problem)

------------------------------------------------------------------
	       SCS v3.2.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 202567, constraints m: 229823
cones: 	  z: primal zero / dual free vars: 27256
	  l: linear vars: 1
	  s: psd vars: 202566, ssize: 1
settings: eps_abs: 1.0e-09, eps_rel: 1.0e-09, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 50000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 600605, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 2.00e+01  1.00e+00  1.99e+01 -1.01e+01  1.00e-01  7.32e-01 
   250| 1.41e-05  2.73e-07  1.36e-05 -5.00e-02  3.17e-01  7

In [57]:
λ, Q = LowCohomologySOS.get_solution(sos_problem)
LowCohomologySOS.certify_sos_decomposition(Δ₃, I, λ, Q, half_basis)

┌ Info: l₁ norm of the error in interval arithmetic:
│   l1_norm = [2.45298e-07, 2.45308e-07]
└   radius(l1_norm) = 4.604003119329727e-12


(false, [-2.44603e-07, -2.44593e-07])

## Δ₄

In [54]:
I = reshape([one(RG_star)],1,1)
sos_problem = LowCohomologySOS.sos_problem(Δ₄, I, 0.05)
JuMP.set_optimizer(sos_problem, scs_opt(eps = 1e-9, max_iters = 20_000))
JuMP.optimize!(sos_problem)

------------------------------------------------------------------
	       SCS v3.2.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 50722, constraints m: 57536
cones: 	  z: primal zero / dual free vars: 6814
	  l: linear vars: 1
	  s: psd vars: 50721, ssize: 1
settings: eps_abs: 1.0e-09, eps_rel: 1.0e-09, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 20000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 148617, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 2.00e+01  1.00e+00  2.00e+01 -1.01e+01  1.00e-01  1.49e-01 
   250| 5.92e-06  5.89e-07  2.77e-06 -5.00e-02  1.00e-01  9.71e

In [55]:
λ, Q = LowCohomologySOS.get_solution(sos_problem)
LowCohomologySOS.certify_sos_decomposition(Δ₄, I, λ, Q, half_basis)

┌ Info: l₁ norm of the error in interval arithmetic:
│   l1_norm = [0.00343223, 0.00343224]
└   radius(l1_norm) = 1.7085048653608936e-12


(true, [0.0173627, 0.0173628])

## Δ₅

In [60]:
I = reshape([one(RG_star)],1,1)
sos_problem = LowCohomologySOS.sos_problem(Δ₅, I, 0.05)
JuMP.set_optimizer(sos_problem, scs_opt(eps = 1e-9, max_iters = 20_000))
JuMP.optimize!(sos_problem)

------------------------------------------------------------------
	       SCS v3.2.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 50722, constraints m: 57536
cones: 	  z: primal zero / dual free vars: 6814
	  l: linear vars: 1
	  s: psd vars: 50721, ssize: 1
settings: eps_abs: 1.0e-09, eps_rel: 1.0e-09, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 20000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 148617, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 2.00e+01  1.00e+00  2.00e+01 -1.01e+01  1.00e-01  1.63e-01 
   250| 6.99e-07  3.08e-07  1.53e-05 -5.00e-02  1.11e+01  1.50e

In [61]:
λ, Q = LowCohomologySOS.get_solution(sos_problem)
LowCohomologySOS.certify_sos_decomposition(Δ₅, I, λ, Q, half_basis)

┌ Info: l₁ norm of the error in interval arithmetic:
│   l1_norm = [1.32979e-10, 1.35709e-10]
└   radius(l1_norm) = 1.364439889036215e-12


(false, [-1.36576e-12, 1.36313e-12])